In [1]:
# Some standard imports
import io
import numpy as np
import onnx
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class Net(nn.Module):

    def __init__(self, input_dim, output_dim):
        """
        input_dim (int): size of the input features
        output_dim (int): size of the output
        """
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 2)
        self.fc2 = torch.nn.Linear(2, output_dim)

    def forward(self, x): # there are different ways of implementing this
        x = self.fc1(x)
        x = nn.functional.sigmoid(x)
        x = self.fc2(x)
        return [x]

In [3]:
def main():
    import numpy as np
    # create instance of Net
    model = Net(2,1)

    #model.state_dict()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], device=device).float()
    y = torch.tensor([[0], [1], [1], [0]], device=device).view(4,1).float()

    # convert numpy array to tensor
    x_tensor = torch.clone(x)
    y_tensor = torch.clone(y)

    # set training mode
    model.train() 
    # In PyTorch, models have a train() method which, somewhat disappointingly, 
    # does NOT perform a training step. Its only purpose is to set the model to training mode. 
    # Why is this important? Some models may use mechanisms like Dropout, for instance, 
    # which have distinct behaviors during training and evaluation phases.


    # set training parameters
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss() # defines a MSE Loss function

    # defines number of epochs
    num_epochs = 50001
    # start to train
    epoch_loss = []
    for epoch in range(num_epochs):
        # forward
        outputs = model(x_tensor)[0]

        # calculate loss
        loss = criterion(outputs, y_tensor)

        # update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # save loss of this epoch
        epoch_loss.append(loss.data.numpy().tolist()) 
        
        if epoch % 1000 == 0:
            print("Epoch: {0}, Loss: {1}, ".format(epoch, loss.to("cpu").detach().numpy()))
    
    print(model.state_dict())

     

In [4]:
if __name__ == "__main__": 

    # Let's build our model 
    main()

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0, Loss: 0.4190930128097534, 
Epoch: 1000, Loss: 0.24957773089408875, 
Epoch: 2000, Loss: 0.24921050667762756, 
Epoch: 3000, Loss: 0.24883157014846802, 
Epoch: 4000, Loss: 0.24838417768478394, 
Epoch: 5000, Loss: 0.24780577421188354, 
Epoch: 6000, Loss: 0.24701914191246033, 
Epoch: 7000, Loss: 0.24592408537864685, 
Epoch: 8000, Loss: 0.24439266324043274, 
Epoch: 9000, Loss: 0.24227480590343475, 
Epoch: 10000, Loss: 0.2394213080406189, 
Epoch: 11000, Loss: 0.23572468757629395, 
Epoch: 12000, Loss: 0.23115622997283936, 
Epoch: 13000, Loss: 0.22576919198036194, 
Epoch: 14000, Loss: 0.21965807676315308, 
Epoch: 15000, Loss: 0.21290390193462372, 
Epoch: 16000, Loss: 0.2055438756942749, 
Epoch: 17000, Loss: 0.197567880153656, 
Epoch: 18000, Loss: 0.18890702724456787, 
Epoch: 19000, Loss: 0.17937445640563965, 
Epoch: 20000, Loss: 0.16857603192329407, 
Epoch: 21000, Loss: 0.1558437943458557, 
Epoch: 22000, Loss: 0.14027494192123413, 
Epoch: 23000, Loss: 0.12101361155509949, 
Epoch: 2400

In [5]:
# PYTORCH FINN-ONNX EXPORT
import torch.onnx 
# set the model to inference mode 
model = Net(2,1)
#model.state_dict()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], device=device).float()
print(x)
y = torch.tensor([[0], [1], [1], [0]], device=device).view(4,1).float()
model.eval() 

# Let's create a dummy input tensor  
dummy_input = x # no test input

# Export the model   
torch.onnx.export(model,         # model being run 
                  dummy_input,       # model input (or a tuple for multiple inputs) 
                 "xor_network.onnx",       # where to save the model  
                 export_params=True,  # store the trained parameter weights inside the model file 
                 opset_version=10,    # the ONNX version to export the model to 
                 do_constant_folding=True,  # whether to execute constant folding for optimization 
                 input_names = ['modelInput'],   # the model's input names 
                 output_names = ['modelOutput'], # the model's output names 
                 dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0: 'batch_size'}}) 

print('Model has been converted to ONNX')

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])
Model has been converted to ONNX


In [6]:
# check the ONNX model with ONNX’s API

onnx_model = onnx.load("xor_network.onnx")
onnx.checker.check_model(onnx_model)

In [7]:
from finn.util.visualization import showSrc, showInNetron
from finn.util.basic import make_build_dir

    
showInNetron("xor_network.onnx")

Serving 'xor_network.onnx' at http://0.0.0.0:8081


In [8]:
from finn.core.modelwrapper import ModelWrapper
model = ModelWrapper("xor_network.onnx")

In [9]:
from finn.core.datatype import DataType

finnonnx_in_tensor_name = model.graph.input[0].name
finnonnx_out_tensor_name = model.graph.output[0].name
print("Input tensor name: %s" % finnonnx_in_tensor_name)
print("Output tensor name: %s" % finnonnx_out_tensor_name)
finnonnx_model_in_shape = model.get_tensor_shape(finnonnx_in_tensor_name)
finnonnx_model_out_shape = model.get_tensor_shape(finnonnx_out_tensor_name)
print("Input tensor shape: %s" % str(finnonnx_model_in_shape))
print("Output tensor shape: %s" % str(finnonnx_model_out_shape))
finnonnx_model_in_dt = model.get_tensor_datatype(finnonnx_in_tensor_name)
finnonnx_model_out_dt = model.get_tensor_datatype(finnonnx_out_tensor_name)
print("Input tensor datatype: %s" % str(finnonnx_model_in_dt.name))
print("Output tensor datatype: %s" % str(finnonnx_model_out_dt.name))
print("List of node operator types in the graph: ")
print([x.op_type for x in model.graph.node])

Input tensor name: modelInput
Output tensor name: modelOutput
Input tensor shape: [0, 2]
Output tensor shape: [0, 1]
Input tensor datatype: FLOAT32
Output tensor datatype: FLOAT32
List of node operator types in the graph: 
['Gemm', 'Sigmoid', 'Gemm']


In [10]:
# Network Preparation

#Tidy up transformations
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.fold_constants import FoldConstants

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save("xor_network.onnx")

In [11]:
showInNetron("xor_network.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network.onnx' at http://0.0.0.0:8081


In [12]:
# Adding Pre and Postprocessing
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType
import brevitas.onnx as bo

model = ModelWrapper("xor_network.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "xor_network_prepoc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

model.save("xor_network_with_prepoc.onnx")
showInNetron("xor_network_with_prepoc.onnx")

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:124: UserWarning: Assuming 2D input is NC
  warnings.warn("Assuming 2D input is NC")


Stopping http://0.0.0.0:8081
Serving 'xor_network_with_prepoc.onnx' at http://0.0.0.0:8081


In [13]:
from finn.transformation.insert_topk import InsertTopK

# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = "xor_network_prepost.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

showInNetron("xor_network_prepost.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network_prepost.onnx' at http://0.0.0.0:8081


In [14]:
# Streamlining
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [15]:
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper("xor_network_prepost.onnx")
# move initial Mul (from preproc) past the Reshape
model = model.transform(MoveScalarLinearPastInvariants())
# streamline
model = model.transform(Streamline())
model.save("xor_network_streamlined.onnx")
showInNetron("xor_network_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network_streamlined.onnx' at http://0.0.0.0:8081


In [16]:
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors

model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(RoundAndClipThresholds())

# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model.save("xor_network_rdy_for_hls.onnx")
showInNetron("xor_network_rdy_for_hls.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network_rdy_for_hls.onnx' at http://0.0.0.0:8081


In [17]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
model = ModelWrapper("xor_network_rdy_for_hls.onnx")
# Mul 
model = model.transform(to_hls.InferLookupLayer())
# TopK to LabelSelect
model = model.transform(to_hls.InferLabelSelectLayer())
model.save("xor_network_hls_layers.onnx")
showInNetron("xor_network_hls_layers.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network_hls_layers.onnx' at http://0.0.0.0:8081


In [20]:
#Dataflow Partition

from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("xor_network_hls_layers.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save("xor_network_dataflow_parent.onnx")
showInNetron("xor_network_dataflow_parent.onnx")

Stopping http://0.0.0.0:8081
Serving 'xor_network_dataflow_parent.onnx' at http://0.0.0.0:8081


In [22]:
# since Dataflow partition node is not created, something might be wrong. 
# I will still continue.
# from finn.custom_op.registry import getCustomOp
# sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
# sdp_node = getCustomOp(sdp_node)
# dataflow_model_filename = sdp_node.get_nodeattr("model")
# showInNetron(dataflow_model_filename)

model = ModelWrapper("xor_network_dataflow_parent.onnx")

In [23]:
fc0 = model.graph.node[0]
fc0w = getCustomOp(fc0)

print("CustomOp wrapper is of class " + fc0w.__class__.__name__)

fc0w.get_nodeattr_types()

ValueError: Empty module name